# Tracking through elements
In this example, we track single particles through different elements and calculate the Jacobian matrix by using PyTorch autograd. We compare the results with bmad_standard, which uses analytic formulas to calculate the Jacobian. We also test functionalities such as element offsets and tilts. As of now, only transverse transformations are supported.

In [1]:
import torch
from torch.autograd.functional import jacobian
from pytao import Tao
import pytao
import time
import os
tkwargs = {
    "dtype" : torch.double
}
torch.set_printoptions(precision= 15, sci_mode=True)
torch.__version__, pytao.__version__

('1.11.0', '0.1.7')

In [2]:
from pathlib import Path

def find_repo(path):
    "Find repository root from the path's parents"
    for path in Path(path).parents:
        # Check whether "path/.git" exists and is a directory
        git_dir = path / ".git"
        if git_dir.is_dir():
            return path

import os
nb_dir = os.getcwd()
# Find the repo root where the script is
repo_path = str(find_repo(nb_dir))
repo_path

'/home/jg2347/Repositories/Bmad-X'

# Constants

In [3]:
from bmadx import M_ELECTRON

# Drift tests

In [4]:
from bmadx import Particle, Drift
from bmadx import make_track_a_drift

In [5]:
# Create drift
L=1.0 # Drift length in m
d1 = Drift(L=torch.tensor(L, **tkwargs)) #named tuple is in track.py module
d1

Drift(L=tensor(1.000000000000000e+00, dtype=torch.float64))

## Drift one particle test

In [6]:
# Incoming particle
s = 0.0 #initial s
p0c = 4.0E+07 #Reference particle momentum in eV
mc2 = 1*M_ELECTRON # electron mass in eV
ts = torch.tensor(s, **tkwargs)
tp0c = torch.tensor(p0c, **tkwargs)
tmc2 = torch.tensor(mc2, **tkwargs)
pvec1 = [2e-3,3e-3,-3e-3,-1e-3,2e-3,-2e-3] 
tvec1 = torch.tensor(pvec1, requires_grad=True, **tkwargs)
p_in = Particle(*tvec1,ts, tp0c, tmc2)
p_in

Particle(x=tensor(2.000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), px=tensor(3.000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), y=tensor(-3.000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), py=tensor(-1.000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), z=tensor(2.000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), pz=tensor(-2.000000000000000e-03, dtype=torch.float64, grad_fn=<UnbindBackward0>), s=tensor(0., dtype=torch.float64), p0c=tensor(4.000000000000000e+07, dtype=torch.float64), mc2=tensor( 5.109989500000000e+05, dtype=torch.float64))

In [7]:
#create track_a_drift_torch
track_a_drift_torch = make_track_a_drift(torch)
# Outgoing particle
p_out = track_a_drift_torch(p_in, d1)
x_py = torch.hstack([p_out.x,p_out.px,p_out.y,p_out.py,p_out.z,p_out.pz]).detach()
x_py

tensor([5.006027114522933e-03, 3.000000000000000e-03, -4.002009038174311e-03,
        -1.000000000000000e-03, 1.994652573892362e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [8]:
# bmad lattice for comparison
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_drift.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out=tao.orbit_at_s(ele=1)
#orbit_out

In [9]:
# bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

tensor([5.006027114522930e-03, 3.000000000000000e-03, -4.002009038174310e-03,
        -1.000000000000000e-03, 1.994652573892360e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [10]:
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

## Drift Jacobian test

In [11]:
# autodiff Jacobian matrix
f_drift = lambda x: track_a_drift_torch(Particle(*x, ts, tp0c, tmc2), d1)[:6]
J = jacobian(f_drift, tvec1)
mat_py = torch.vstack(J)
mat_py

tensor([[ 1.000000000000000e+00,  1.002018092527393e+00,  0.000000000000000e+00,
         -3.018117694005117e-06,  0.000000000000000e+00, -3.012081458617107e-03],
        [ 0.000000000000000e+00,  1.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [ 0.000000000000000e+00, -3.018117694005117e-06,  1.000000000000000e+00,
          1.002010044213542e+00,  0.000000000000000e+00,  1.004027152872369e-03],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          1.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [ 0.000000000000000e+00, -3.012081458617106e-03,  0.000000000000000e+00,
          1.004027152872369e-03,  1.000000000000000e+00,  1.742165247481025e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.000000000000000e+00]],
       dtype=torch.fl

In [12]:
# Bmad formula Jacobian
drift_tao = tao.matrix(0,1)
mat_tao = torch.tensor(drift_tao['mat6'], **tkwargs)
mat_tao

tensor([[ 1.000000000000000e+00,  1.002018092527390e+00,  0.000000000000000e+00,
         -3.018117694005120e-06,  0.000000000000000e+00, -3.012081458617110e-03],
        [ 0.000000000000000e+00,  1.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [ 0.000000000000000e+00, -3.018117694005120e-06,  1.000000000000000e+00,
          1.002010044213540e+00,  0.000000000000000e+00,  1.004027152872370e-03],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          1.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [ 0.000000000000000e+00, -3.012081458617110e-03,  0.000000000000000e+00,
          1.004027152872370e-03,  1.000000000000000e+00,  1.742165247481030e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.000000000000000e+00]],
       dtype=torch.fl

In [13]:
# is it close to Tao result?
torch.allclose(mat_py, mat_tao, atol=0, rtol=1.0e-14)

True

# Quadrupole tests

In [14]:
from bmadx import Quadrupole
from bmadx import make_track_a_quadrupole

In [15]:
def torch_quadrupole(L: torch.Tensor, K1: torch.Tensor, NUM_STEPS=1,
                     X_OFFSET: torch.Tensor=torch.tensor(0.0,**tkwargs),
                     Y_OFFSET: torch.Tensor=torch.tensor(0.0,**tkwargs),
                     TILT: torch.Tensor=torch.tensor(0.0,**tkwargs)):
    return Quadrupole(L=L, K1=K1, NUM_STEPS=NUM_STEPS, X_OFFSET=X_OFFSET,
                      Y_OFFSET=Y_OFFSET, TILT=TILT)

In [16]:
# Create quad
L = 0.1 #Length in m
K1 = 10 #Quad focusing strength. Positive is focusing in x
#NUM_STEPS = 1 #number of divisions for tracking. 1 is bmad default when there are no other multipoles
q1 = torch_quadrupole(L=torch.tensor(L, **tkwargs), K1=torch.tensor(K1, **tkwargs))
q1

Quadrupole(L=tensor(1.000000000000000e-01, dtype=torch.float64), K1=tensor(1.000000000000000e+01, dtype=torch.float64), NUM_STEPS=1, X_OFFSET=tensor(0., dtype=torch.float64), Y_OFFSET=tensor(0., dtype=torch.float64), TILT=tensor(0., dtype=torch.float64))

## Quadrupole one particle test

In [17]:
# create track_a_quadrupole_torch
track_a_quadrupole_torch = make_track_a_quadrupole(torch)
# Outgoing particle
p_out = track_a_quadrupole_torch(p_in, q1)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.196239719302552e-03, 8.841834264853368e-04, -3.253441973269224e-03,
        -4.100871741572859e-03, 1.999394664225331e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [18]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_quad.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [19]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

tensor([2.196239719302550e-03, 8.841834264853370e-04, -3.253441973269220e-03,
        -4.100871741572860e-03, 1.999394664225330e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [20]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

## Quadrupole Jacobian test

In [21]:
# autodiff Jacobian matrix
f_quadrupole = lambda x: track_a_quadrupole_torch(Particle(*x,ts, tp0c, tmc2), q1)[:6]
J = jacobian(f_quadrupole, tvec1)
mat_py = torch.vstack(J)
mat_py

tensor([[ 9.503167431875498e-01,  9.853541097581728e-02,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00, -1.924858550317723e-04],
        [-9.833834015386563e-01,  9.503167431875498e-01, -0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.149663908944082e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  1.050519938506054e+00,
          1.018821577510623e-01,  0.000000000000000e+00,  2.569093937337833e-04],
        [-0.000000000000000e+00,  0.000000000000000e+00,  1.016783934355602e+00,
          1.050519938506054e+00,  0.000000000000000e+00,  1.017485822657404e-04],
        [ 8.003290869842023e-05, -1.942507914386516e-04,  1.543324297486644e-04,
          2.595220753974964e-04,  1.000000000000000e+00,  1.756709202142694e-05],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.000000000000000e+00]],
       dtype=torch.fl

In [22]:
# Bmad formula Jacobian
quad_tao = tao.matrix(0,1)
mat_tao = torch.tensor(quad_tao['mat6'], **tkwargs)
mat_tao

tensor([[ 9.503167431875500e-01,  9.853541097581731e-02,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00, -1.924858550317720e-04],
        [-9.833834015386560e-01,  9.503167431875500e-01,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.149663908944080e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  1.050519938506050e+00,
          1.018821577510620e-01,  0.000000000000000e+00,  2.569093937337830e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  1.016783934355600e+00,
          1.050519938506050e+00,  0.000000000000000e+00,  1.017485822657410e-04],
        [ 8.003290869842020e-05, -1.942507914386520e-04,  1.543324297486640e-04,
          2.595220753974960e-04,  1.000000000000000e+00,  1.756709202142690e-05],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.000000000000000e+00]],
       dtype=torch.fl

In [23]:
# close to Tao result?
torch.allclose(mat_py, mat_tao, atol=0, rtol=1.0e-14)

True

## Quadrupole offset test

In [24]:
# Quad params
L = 0.1 #Length in m
K1 = 10 #Quad focusing strength. Positive is focusing in x

# quad w/o offsets
q_no_off = torch_quadrupole(L=torch.tensor(L, **tkwargs), K1=torch.tensor(K1, **tkwargs))
# quad with offsets
x_off = 1e-3
y_off = -2e-3
q_off = torch_quadrupole(L=torch.tensor(L, **tkwargs), K1=torch.tensor(K1, **tkwargs), 
                         X_OFFSET=torch.tensor(x_off, **tkwargs),
                         Y_OFFSET=torch.tensor(y_off, **tkwargs) )

In [25]:
# Outgoing particle no offset
p_out = track_a_quadrupole_torch(p_in, q_no_off)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.196239719302552e-03, 8.841834264853368e-04, -3.253441973269224e-03,
        -4.100871741572859e-03, 1.999394664225331e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [26]:
# Outgoing particle offset
p_out = track_a_quadrupole_torch(p_in, q_off)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.245922976115002e-03, 1.867566828023993e-03, -3.152402096257116e-03,
        -2.067303872861656e-03, 1.999538606428305e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [27]:
%%time
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_quad_offset.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

CPU times: user 42.6 ms, sys: 7.58 ms, total: 50.1 ms
Wall time: 74.4 ms


In [28]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

tensor([2.245922976115000e-03, 1.867566828023990e-03, -3.152402096257120e-03,
        -2.067303872861660e-03, 1.999538606428300e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [29]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

## Quadrupole tilt test (transverse rotation)

In [30]:
# Quad params
L = 0.1 #Length in m
K1 = 10 #Quad focusing strength. Positive is focusing in x

# quad w/o tilt
q_no_tilt = torch_quadrupole(L=torch.tensor(L, **tkwargs), K1=torch.tensor(K1, **tkwargs))
# quad with tilt
tilt = 0.3
q_tilt = torch_quadrupole(L=torch.tensor(L, **tkwargs), K1=torch.tensor(K1, **tkwargs),
                          TILT = torch.tensor(tilt, **tkwargs))

In [31]:
# Outgoing particle no tilt
p_out = track_a_quadrupole_torch(p_in, q_no_tilt)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.196239719302552e-03, 8.841834264853368e-04, -3.253441973269224e-03,
        -4.100871741572859e-03, 1.999394664225331e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [32]:
# Outgoing particle tilt
p_out = track_a_quadrupole_torch(Particle(*tvec1,ts, tp0c, tmc2), q_tilt)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.300431812756860e-03, 2.982152956166943e-03, -3.286310353406088e-03,
        -4.782331681936468e-03, 1.999046404985869e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [33]:
%%time
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_quad_tilt.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

CPU times: user 25.8 ms, sys: 59.7 ms, total: 85.5 ms
Wall time: 230 ms


In [34]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

tensor([2.300431812756860e-03, 2.982152956166940e-03, -3.286310353406090e-03,
        -4.782331681936470e-03, 1.999046404985870e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [35]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

# Crab_Cavity tests

In [36]:
from bmadx import CrabCavity
from bmadx import make_track_a_crab_cavity

In [37]:
def torch_crab_cavity(L: torch.Tensor, VOLTAGE: torch.Tensor,
                      PHI0: torch.Tensor, RF_FREQUENCY: torch.Tensor,
                      X_OFFSET: torch.Tensor=torch.tensor(0.0,**tkwargs),
                      Y_OFFSET: torch.Tensor=torch.tensor(0.0,**tkwargs),
                      TILT: torch.Tensor=torch.tensor(0.0,**tkwargs)):
    return CrabCavity(L=L, VOLTAGE=VOLTAGE, PHI0=PHI0,
                      RF_FREQUENCY=RF_FREQUENCY, X_OFFSET=X_OFFSET,
                      Y_OFFSET=Y_OFFSET, TILT=TILT)

In [38]:
# Create cavity
L = 0.2
voltage = 1.0e4
phi0 = 0.5
rf = 1.0e9
cav = torch_crab_cavity(L=torch.tensor(L, **tkwargs),
                        VOLTAGE=torch.tensor(voltage, **tkwargs),
                        PHI0=torch.tensor(phi0, **tkwargs),
                        RF_FREQUENCY=torch.tensor(rf, **tkwargs))
cav

CrabCavity(L=tensor(2.000000000000000e-01, dtype=torch.float64), VOLTAGE=tensor(1.000000000000000e+04, dtype=torch.float64), PHI0=tensor(5.000000000000000e-01, dtype=torch.float64), RF_FREQUENCY=tensor(1.000000000000000e+09, dtype=torch.float64), X_OFFSET=tensor(0., dtype=torch.float64), Y_OFFSET=tensor(0., dtype=torch.float64), TILT=tensor(0., dtype=torch.float64))

## Crab_cavity one particle test

In [39]:
# create track_a_crab_cavity_torch
track_a_crab_cavity_torch = make_track_a_crab_cavity(torch)
# Outgoing particle
p_out = track_a_crab_cavity_torch(p_in, cav)
#p_out
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.600159503009856e-03, 2.989525785200116e-03, -3.200403013811110e-03,
        -1.000000000000000e-03, 1.998933450470348e-03, -2.012044669173099e-03],
       dtype=torch.float64)

In [40]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_crab_cavity.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [41]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

tensor([2.600159503009860e-03, 2.989525785200120e-03, -3.200403013811110e-03,
        -1.000000000000000e-03, 1.998933450470350e-03, -2.012044669173100e-03],
       dtype=torch.float64)

In [42]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

## Crab_cavity Jacobian test

In [43]:
# autodiff Jacobian matrix
f_crab_cavity = lambda x: track_a_crab_cavity_torch(Particle(*x,ts, tp0c, tmc2), cav)[:6]
J = jacobian(f_crab_cavity, tvec1)
mat_py = torch.vstack(J)
mat_py

tensor([[ 1.000001571489388e+00,  2.004051338856653e-01,  0.000000000000000e+00,
         -6.552534052264527e-07, -5.246100940167111e-04, -6.013813103820471e-04],
        [ 0.000000000000000e+00,  1.000001576957398e+00,  0.000000000000000e+00,
         -5.256524659788450e-07, -5.235440739575950e-03, -8.949162872176157e-08],
        [-5.256651057680826e-07, -6.552540421507948e-07,  1.000000000000000e+00,
          2.004032150230133e-01,  2.637597029868803e-09,  2.008080032283205e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          1.000000000000000e+00, -0.000000000000000e+00,  0.000000000000000e+00],
        [-9.289874897604998e-08, -6.013816504874850e-04, -0.000000000000000e+00,
          2.008080022802770e-04,  1.000001569699991e+00,  3.483769060576969e-05],
        [-5.235440749927688e-03, -5.246038241196926e-04, -0.000000000000000e+00,
          2.643031443043208e-09,  1.058650459477272e-05,  1.000001579115604e+00]],
       dtype=torch.fl

In [44]:
# Bmad formula Jacobian
cav_tao = tao.matrix(0,1)
mat_tao = torch.tensor(cav_tao['mat6'], **tkwargs)
mat_tao

tensor([[ 1.000001571489390e+00,  2.004051338856650e-01,  0.000000000000000e+00,
         -6.552534052264531e-07, -5.246100940167110e-04, -6.013813103820470e-04],
        [ 0.000000000000000e+00,  1.000001576957400e+00,  0.000000000000000e+00,
         -5.256524659788450e-07, -5.235440739575950e-03, -8.949162872175810e-08],
        [-5.256651057680830e-07, -6.552540421507950e-07,  1.000000000000000e+00,
          2.004032150230130e-01,  2.637597029868800e-09,  2.008080032283200e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          1.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [-9.289874897604990e-08, -6.013816504874850e-04,  0.000000000000000e+00,
          2.008080022802770e-04,  1.000001569699990e+00,  3.483769060576920e-05],
        [-5.235440749927690e-03, -5.246038241196930e-04,  0.000000000000000e+00,
          2.643031443043210e-09,  1.058650459477270e-05,  1.000001579115600e+00]],
       dtype=torch.fl

In [45]:
# close to Tao result up to 13th decimal place
torch.allclose(mat_py, mat_tao, atol=0, rtol=1.0e-13)

True

In [46]:
# but not up to 14th decimal place
torch.isclose(mat_py, mat_tao, atol=0, rtol=1e-14)

tensor([[ True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True]])

## Crab_cavity offset test

In [47]:
# Cavity params
L = 0.2
voltage = 1.0e3
phi0 = 0.5
rf = 1.0e9

# cav w/o offsets
cav_no_off = torch_crab_cavity(L=torch.tensor(L, **tkwargs),
                               VOLTAGE=torch.tensor(voltage, **tkwargs),
                               PHI0=torch.tensor(phi0, **tkwargs),
                               RF_FREQUENCY=torch.tensor(rf, **tkwargs))

#cav with offsets
x_off = 1e-3
y_off = -2e-3
cav_off = torch_crab_cavity(L=torch.tensor(L, **tkwargs),
                            VOLTAGE=torch.tensor(voltage, **tkwargs),
                            PHI0=torch.tensor(phi0, **tkwargs),
                            RF_FREQUENCY=torch.tensor(rf, **tkwargs),
                            X_OFFSET=torch.tensor(x_off, **tkwargs),
                            Y_OFFSET=torch.tensor(y_off, **tkwargs))

In [48]:
# Outgoing particle no offset
p_out = track_a_crab_cavity_torch(p_in, cav_no_off)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.601100832046753e-03, 2.998952578520011e-03, -3.200401928250687e-03,
        -1.000000000000000e-03, 1.998930808836860e-03, -2.001204466916434e-03],
       dtype=torch.float64)

In [49]:
# Outgoing particle offset
p_out = track_a_crab_cavity_torch(Particle(*tvec1,ts, tp0c, tmc2), cav_off)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.601100674405775e-03, 2.998952578520011e-03, -3.200401875685341e-03,
        -1.000000000000000e-03, 1.998930818129399e-03, -2.000680922842398e-03],
       dtype=torch.float64)

In [50]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_crab_cavity_offset.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [51]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

tensor([2.601100674405770e-03, 2.998952578520010e-03, -3.200401875685340e-03,
        -1.000000000000000e-03, 1.998930818129400e-03, -2.000680922842400e-03],
       dtype=torch.float64)

In [52]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

## Crab_cavity tilt test

In [53]:
# Cavity params
L = 0.2 #Length in m
voltage = 1.0e3 #Quad focusing strength. Positive is focusing in x
phi0 = 0.5
rf = 1.0e9

# cav w/o tilt
cav_no_tilt = torch_crab_cavity(L=torch.tensor(L, **tkwargs),
                                VOLTAGE=torch.tensor(voltage, **tkwargs),
                                PHI0=torch.tensor(phi0, **tkwargs),
                                RF_FREQUENCY=torch.tensor(rf, **tkwargs))

#cav with tilt
x_off = 1e-3
y_off = -2e-3
tilt = 0.3
cav_tilt = torch_crab_cavity(L=torch.tensor(L, **tkwargs),
                             VOLTAGE=torch.tensor(voltage, **tkwargs),
                             PHI0=torch.tensor(phi0, **tkwargs),
                             RF_FREQUENCY=torch.tensor(rf, **tkwargs),
                             TILT=torch.tensor(tilt, **tkwargs))

In [54]:
# Outgoing particle no offset
p_out = track_a_crab_cavity_torch(p_in, cav_no_tilt)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.601100832046753e-03, 2.998952578520011e-03, -3.200401928250687e-03,
        -1.000000000000000e-03, 1.998930808836860e-03, -2.001204466916434e-03],
       dtype=torch.float64)

In [55]:
# Outgoing particle offset and tilt
p_out = track_a_crab_cavity_torch(Particle(*tvec1,ts, tp0c, tmc2), cav_tilt)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.601105359111942e-03, 2.998999360040673e-03, -3.200432890364399e-03,
        -1.000309534212228e-03, 1.998930773136149e-03, -2.000671014767513e-03],
       dtype=torch.float64)

In [56]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_crab_cavity_tilt.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [57]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

tensor([2.601105359111940e-03, 2.998999360040670e-03, -3.200432890364400e-03,
        -1.000309534212230e-03, 1.998930773136150e-03, -2.000671014767510e-03],
       dtype=torch.float64)

In [58]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

In [59]:
f_crab_cavity = lambda x: track_a_crab_cavity_torch(Particle(*x,ts, tp0c, tmc2), cav_tilt)[:6]
J = jacobian(f_crab_cavity, tvec1)
mat_py = torch.vstack(J)
mat_py

tensor([[ 1.000000150602446e+00,  2.004037151906528e-01,  4.658679586544423e-08,
         -6.039807690927379e-07, -5.011717697334090e-05, -6.023171246175704e-04],
        [ 0.000000000000000e+00,  1.000000150652494e+00,  0.000000000000000e+00,
         -5.021749810429910e-08, -5.001607575171654e-04, -8.549461838918098e-09],
        [-5.023310933882641e-08, -6.039807756291760e-07,  9.999999844610783e-01,
          2.004020728926290e-01, -1.550268447215676e-05,  2.008363914804506e-04],
        [ 0.000000000000000e+00,  4.660227759245572e-08,  0.000000000000000e+00,
          9.999999844659074e-01, -1.547178529322681e-04, -2.644658461431463e-09],
        [-8.877538963465590e-09, -6.023171574216554e-04, -2.746144608721748e-09,
          2.008363812799022e-04,  1.000000134963841e+00,  3.484280382627325e-05],
        [-5.001607575722588e-04, -5.011714377673051e-05, -1.547178529493105e-04,
         -1.550267409755359e-05,  5.897796618608915e-07,  1.000000135238635e+00]],
       dtype=torch.fl

In [60]:
cav_tao = tao.matrix(0,1)
mat_tao = torch.tensor(cav_tao['mat6'], **tkwargs)
mat_tao

tensor([[ 1.000000150602450e+00,  2.004037151906530e-01,  4.658679586544420e-08,
         -6.039807690857990e-07, -5.011717697334090e-05, -6.023171246175700e-04],
        [ 0.000000000000000e+00,  1.000000150652490e+00,  0.000000000000000e+00,
         -5.021749810429910e-08, -5.001607575171650e-04, -8.549461838917659e-09],
        [-5.023310933882640e-08, -6.039807756222370e-07,  9.999999844610780e-01,
          2.004020728926290e-01, -1.550268447215680e-05,  2.008363914804510e-04],
        [ 0.000000000000000e+00,  4.660227764796690e-08,  0.000000000000000e+00,
          9.999999844659070e-01, -1.547178529322680e-04, -2.644658461431380e-09],
        [-8.877538963465760e-09, -6.023171574216550e-04, -2.746144608721800e-09,
          2.008363812799020e-04,  1.000000134963840e+00,  3.484280382627260e-05],
        [-5.001607575722590e-04, -5.011714377673050e-05, -1.547178529493100e-04,
         -1.550267409755360e-05,  5.897796618608910e-07,  1.000000135238630e+00]],
       dtype=torch.fl

In [61]:
torch.isclose(mat_py, mat_tao, atol=0, rtol=1.0e-10)

tensor([[ True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True],
        [ True, False,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True]])

In [62]:
# Something is wrong with the Jacobian matrix of a tilted cavity. Currently debuging.

# RF_Cavity tests

In [63]:
from bmadx import RFCavity
from bmadx import make_track_a_rf_cavity

In [64]:
def torch_rf_cavity(L: torch.Tensor, VOLTAGE: torch.Tensor,
                    PHI0: torch.Tensor, RF_FREQUENCY: torch.Tensor,
                    X_OFFSET: torch.Tensor=torch.tensor(0.0,**tkwargs),
                    Y_OFFSET: torch.Tensor=torch.tensor(0.0,**tkwargs),
                    TILT: torch.Tensor=torch.tensor(0.0,**tkwargs)):
    return RFCavity(L=L, VOLTAGE=VOLTAGE, PHI0=PHI0,
                    RF_FREQUENCY=RF_FREQUENCY, X_OFFSET=X_OFFSET,
                    Y_OFFSET=Y_OFFSET, TILT=TILT)

In [65]:
# Create cavity
L = 0.2
voltage = 1.0e4
phi0 = 0.5
rf = 1.0e9
cav = torch_rf_cavity(L=torch.tensor(L, **tkwargs),
                      VOLTAGE=torch.tensor(voltage, **tkwargs),
                      PHI0=torch.tensor(phi0, **tkwargs),
                      RF_FREQUENCY=torch.tensor(rf, **tkwargs))
cav

RFCavity(L=tensor(2.000000000000000e-01, dtype=torch.float64), VOLTAGE=tensor(1.000000000000000e+04, dtype=torch.float64), PHI0=tensor(5.000000000000000e-01, dtype=torch.float64), RF_FREQUENCY=tensor(1.000000000000000e+09, dtype=torch.float64), X_OFFSET=tensor(0., dtype=torch.float64), Y_OFFSET=tensor(0., dtype=torch.float64), TILT=tensor(0., dtype=torch.float64))

## RF_Cavity one particle test

In [66]:
# create track_a_rf_cavity_torch
track_a_rf_cavity_torch = make_track_a_rf_cavity(torch)
# Outgoing particle
p_out = track_a_rf_cavity_torch(p_in, cav)
#p_out
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.601208578941774e-03, 3.000000000000000e-03, -3.200402859647258e-03,
        -1.000000000000000e-03, 1.998930328796947e-03, -2.010475072419949e-03],
       dtype=torch.float64)

In [67]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_rf_cavity.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [68]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

tensor([2.601208578941770e-03, 3.000000000000000e-03, -3.200402859647260e-03,
        -1.000000000000000e-03, 1.998930328796950e-03, -2.010475072419950e-03],
       dtype=torch.float64)

In [69]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

## RF_Cavity Jacobian test

In [70]:
# autodiff Jacobian matrix
f_rf_cavity = lambda x: track_a_rf_cavity_torch(Particle(*x,ts, tp0c, tmc2), cav)[:6]
J = jacobian(f_rf_cavity, tvec1)
mat_py = torch.vstack(J)
mat_py

tensor([[ 1.000000000000000e+00,  2.004046705463930e-01,  0.000000000000000e+00,
         -6.036330450442983e-07,  1.577102406617731e-06, -6.024226175949689e-04],
        [ 0.000000000000000e+00,  1.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [ 0.000000000000000e+00, -6.036330450442982e-07,  1.000000000000000e+00,
          2.004030608582729e-01, -5.257008022059102e-07,  2.008075391983230e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          1.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [ 0.000000000000000e+00, -6.024226155237540e-04,  0.000000000000000e+00,
          2.008075385079180e-04,  9.999999053429351e-01,  3.484386392945851e-05],
        [ 0.000000000000000e+00,  1.577103892075475e-06,  0.000000000000000e+00,
         -5.257012973584918e-07, -5.235869418127461e-03,  9.999999122191349e-01]],
       dtype=torch.fl

In [71]:
# Bmad formula Jacobian
quad_tao = tao.matrix(0,1)
mat_tao = torch.tensor(quad_tao['mat6'], **tkwargs)
mat_tao

tensor([[ 1.000000000000000e+00,  2.004046705463930e-01,  0.000000000000000e+00,
         -6.036330450442980e-07,  1.577102406617730e-06, -6.024226175949690e-04],
        [ 0.000000000000000e+00,  1.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [ 0.000000000000000e+00, -6.036330450442980e-07,  1.000000000000000e+00,
          2.004030608582730e-01, -5.257008022059100e-07,  2.008075391983230e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          1.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [ 0.000000000000000e+00, -6.024226155237540e-04,  0.000000000000000e+00,
          2.008075385079180e-04,  9.999999053429350e-01,  3.484386392945810e-05],
        [ 0.000000000000000e+00,  1.577103892075480e-06,  0.000000000000000e+00,
         -5.257012973584920e-07, -5.235869418127460e-03,  9.999999122191350e-01]],
       dtype=torch.fl

In [72]:
# close to Tao result?
torch.allclose(mat_py, mat_tao, atol=0, rtol=1.0e-13)

True

## RF_Cavity offset test

In [73]:
# Cavity params
L = 0.2
voltage = 1.0e3
phi0 = 0.5
rf = 1.0e9

# cav w/o offsets
cav_no_off = torch_rf_cavity(L=torch.tensor(L, **tkwargs),
                               VOLTAGE=torch.tensor(voltage, **tkwargs),
                               PHI0=torch.tensor(phi0, **tkwargs),
                               RF_FREQUENCY=torch.tensor(rf, **tkwargs))

# cav with offsets
x_off = 1e-3
y_off = -2e-3
cav_off = torch_rf_cavity(L=torch.tensor(L, **tkwargs),
                          VOLTAGE=torch.tensor(voltage, **tkwargs),
                          PHI0=torch.tensor(phi0, **tkwargs),
                          RF_FREQUENCY=torch.tensor(rf, **tkwargs),
                          X_OFFSET=torch.tensor(x_off, **tkwargs),
                          Y_OFFSET=torch.tensor(y_off, **tkwargs))

In [74]:
# Outgoing particle no offset
p_out = track_a_rf_cavity_torch(p_in, cav_no_off)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.601205738506814e-03, 3.000000000000000e-03, -3.200401912835605e-03,
        -1.000000000000000e-03, 1.998930496180454e-03, -2.001047507284194e-03],
       dtype=torch.float64)

In [75]:
# Outgoing particle offset
p_out = track_a_rf_cavity_torch(Particle(*tvec1,ts, tp0c, tmc2), cav_off)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.601205738506814e-03, 3.000000000000000e-03, -3.200401912835605e-03,
        -1.000000000000000e-03, 1.998930496180454e-03, -2.001047507284194e-03],
       dtype=torch.float64)

In [76]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_rf_cavity_offset.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [77]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

tensor([2.601205738506810e-03, 3.000000000000000e-03, -3.200401912835600e-03,
        -1.000000000000000e-03, 1.998930496180450e-03, -2.001047507284190e-03],
       dtype=torch.float64)

In [78]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

## RF_Cavity tilt test

In [79]:
# Cavity params
L = 0.2 #Length in m
voltage = 1.0e3 #Quad focusing strength. Positive is focusing in x
phi0 = 0.5
rf = 1.0e9

# cav w/o tilt
cav_no_tilt = torch_rf_cavity(L=torch.tensor(L, **tkwargs),
                              VOLTAGE=torch.tensor(voltage, **tkwargs),
                              PHI0=torch.tensor(phi0, **tkwargs),
                              RF_FREQUENCY=torch.tensor(rf, **tkwargs))

#cav with tilt
x_off = 1e-3
y_off = -2e-3
tilt = 0.3
cav_tilt = torch_rf_cavity(L=torch.tensor(L, **tkwargs),
                           VOLTAGE=torch.tensor(voltage, **tkwargs),
                           PHI0=torch.tensor(phi0, **tkwargs),
                           RF_FREQUENCY=torch.tensor(rf, **tkwargs),
                           TILT=torch.tensor(tilt, **tkwargs))

In [80]:
# Outgoing particle no offset
p_out = track_a_rf_cavity_torch(p_in, cav_no_tilt)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.601205738506814e-03, 3.000000000000000e-03, -3.200401912835605e-03,
        -1.000000000000000e-03, 1.998930496180454e-03, -2.001047507284194e-03],
       dtype=torch.float64)

In [81]:
# Outgoing particle offset and tilt
p_out = track_a_rf_cavity_torch(Particle(*tvec1,ts, tp0c, tmc2), cav_tilt)
x_py = torch.hstack(p_out[:6]).detach()
x_py

tensor([2.601205738506814e-03, 3.000000000000000e-03, -3.200401912835605e-03,
        -1.000000000000000e-03, 1.998930496180454e-03, -2.001047507284194e-03],
       dtype=torch.float64)

In [82]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_rf_cavity_tilt.bmad -noplot')
tao.cmd('set particle_start x='+str(pvec1[0]))
tao.cmd('set particle_start px='+str(pvec1[1]))
tao.cmd('set particle_start y='+str(pvec1[2]))
tao.cmd('set particle_start py='+str(pvec1[3]))
tao.cmd('set particle_start z='+str(pvec1[4]))
tao.cmd('set particle_start pz='+str(pvec1[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [83]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],orbit_out['px'],orbit_out['y'],orbit_out['py'],orbit_out['z'],orbit_out['pz']],**tkwargs)
x_tao

tensor([2.601205738506810e-03, 3.000000000000000e-03, -3.200401912835600e-03,
        -1.000000000000000e-03, 1.998930496180450e-03, -2.001047507284190e-03],
       dtype=torch.float64)

In [84]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True